In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = 'data/train_data.csv'
sample_submission = '/AI_Challenge/sample_submission.csv'

In [ ]:
train_data = pd.read_csv(train_data)
# sample_submission = pd.read_csv(sample_submission)

In [ ]:
train_data = train_data.fillna(method = 'bfill')

In [ ]:
train_data.head()

,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1.0,1.0,00:00,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66
1,1.0,1.0,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66
2,1.0,1.0,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76
3,1.0,1.0,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53
4,1.0,1.0,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36


In [ ]:
##실제 성능이 좋은 모델은 해당 아래 to-do에 전처리를 안한 모델이므로 주석처리 하였습니다...

Ndir_mean = train_data['Ndir'].mean()
Wdir_mean = train_data['Wdir'].mean()
Wspd_mean = train_data['Wspd'].mean()
Pab1_mean = train_data['Pab1'].mean()
Pab2_mean = train_data['Pab2'].mean()
Pab3_mean = train_data['Pab3'].mean()

In [ ]:
train_data.loc[(train_data['Ndir'] > 720) | (train_data['Ndir'] < -720),'Ndir'] = Ndir_mean
train_data.loc[(train_data['Wdir'] > 180) | (train_data['Wdir'] < -180),'Wdir'] = Wdir_mean
train_data.loc[(train_data['Patv'] <= 0) & (train_data['Wspd'] > 2.5),'Wspd'] = Wspd_mean

train_data.loc[(train_data['Pab1'] > 89), 'Pab1'] = Pab1_mean
train_data.loc[ (train_data['Pab2'] > 89), 'Pab2'] = Pab2_mean
train_data.loc[(train_data['Pab3'] > 89), 'Pab3'] = Pab3_mean

In [ ]:
pram = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':128,
    'SEED':42
}

In [ ]:
def makeing_train(data):
    train_x, train_y = [], []
    for i in tqdm(sorted(pd.unique(data["TurbID"]))):
        tmp_data = data[data["TurbID"] == i]
        for j in range(1, 223 - 6):
            
            day_list = [x for x in range(j, j+ 5)]
            label_day_list = [y for y in range(j+5, j + 7)]
            
            train_tmp = tmp_data[tmp_data["Day"].isin(day_list)]
            label_tmp = tmp_data[tmp_data["Day"].isin(label_day_list)]["Patv"]
            
            # feature 선택 및 제거
            train_tmp = train_tmp.drop(["TurbID", "Day"], axis = 1)
            
            train_x.append(np.array(train_tmp))
            train_y.append(np.array(label_tmp))
            
    return train_x, train_y

In [ ]:
tms_list = list(pd.unique(train_data["Tmstamp"]))
train_data["Tmstamp"] = train_data["Tmstamp"].apply(lambda x : tms_list.index(x) + 1)

100%|████████████████████████████████████████████████████████████████████████████████| 134/134 [00:36<00:00,  3.66it/s]


In [ ]:
train_x, train_y = makeing_train(train_data)

In [ ]:
train_x = np.array(train_x).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
train_y = np.array(train_y)

In [ ]:
train_x.shape, train_y.shape

((25996, 720, 11), (25996, 288))

In [ ]:
df.dropna(inplace = True)

In [ ]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1:]

# test_x = df_test.iloc[:,0:-1]
# test_y = df_test.iloc[:,-1:]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
class Totensor(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.LongTensor(self.X[index]), torch.LongTensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [ ]:
train_dataset = Totensor(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size = pram['BATCH_SIZE'], shuffle=True, num_workers=0)

In [ ]:
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision.models import resnet18, resnet101
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F


class DNN(nn.Module): #Train Loss : [152984.89595] Train MAE : [306.38701]
    """Deep Neural Network"""
    def __init__(self, input_size, hidden_size, output_size):
        super(DNN, self).__init__()

        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = x.squeeze(dim=2)
        out = self.main(x[:,-1,:])
        return out


class GRU(nn.Module): 
    """Gat e Recurrent Unit"""
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRU, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.gru = nn.GRU(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)

        return out

# 모델 학습

def train(model, optimizer, train_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    metric = nn.L1Loss().to(device)
    best_mae = 9999999
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.type(torch.FloatTensor)
            Y = Y.type(torch.FloatTensor)
            
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            with torch.no_grad():
                mae = metric(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_mae.append(mae.item())
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Train MAE : [{np.mean(train_mae):.5f}]')
        
        if best_mae > np.mean(train_mae):
            best_mae = np.mean(train_mae)
            torch.save(model.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
            print('Model Saved.')

In [ ]:
model = GRU(11,516,1,288)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = pram["LEARNING_RATE"])
train(model, optimizer, train_loader, device)

100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:26<00:00,  7.71it/s]


Epoch : [1] Train Loss : [308347.42272] Train MAE : [357.30609]
Model Saved.


100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:25<00:00,  7.91it/s]


Epoch : [2] Train Loss : [304353.97633] Train MAE : [354.83846]
Model Saved.


100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:25<00:00,  7.89it/s]


...

Epoch : [8] Train Loss : [284049.35409] Train MAE : [343.47206]
Model Saved.


100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:25<00:00,  7.93it/s]


Epoch : [9] Train Loss : [280608.76256] Train MAE : [341.50772]
Model Saved.


100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:26<00:00,  7.84it/s]

Epoch : [10] Train Loss : [277797.56150] Train MAE : [339.95311]
Model Saved.


In [ ]:
test_data_list = [x for x in range(218, 223)]

test_data = train_data[train_data["Day"].isin(test_data_list)]

test_data = test_data.drop(["TurbID", "Day"], axis = 1)

test_data = np.array(test_data).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])

In [ ]:
test_dataset = Totensor(test_data, None)
test_loader = DataLoader(test_dataset, batch_size = pram['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            pred = model(X)
            preds += pred.cpu().tolist()
    return np.array(preds)

In [ ]:
model = GRU(11,516,1,288)
best_checkpoint = torch.load('./best_model.pth')
model.load_state_dict(best_checkpoint)
model.eval()

In [ ]:
preds = predict(model, test_loader, device)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.92it/s]


In [ ]:
preds = preds.reshape(-1)

In [ ]:
preds

array([51.73679733, 53.09480286, 53.62515259, ..., 52.10406113,
       52.56267166, 53.28595734])

In [ ]:
sample_submission["Patv"] = preds
sample_submission.to_csv("./GRU0729.csv", index = False)